In [2]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.library.http import NBAStatsHTTP
import pandas as pd
import time

# Extend timeout to prevent timeout errors
NBAStatsHTTP.TIMEOUT = 60

# Settings
MAX_PLAYERS = None   # Set to None to fetch all players
MAX_RETRIES = 3

# Retry-safe API caller
def retry_request(func, **kwargs):
    for attempt in range(MAX_RETRIES):
        try:
            return func(**kwargs)
        except Exception as e:
            print(f"⚠️ Attempt {attempt+1} failed: {e}")
            time.sleep(2)
    return None

# Start processing
all_players = players.get_active_players()
stats_data = []

for i, p in enumerate(all_players[:MAX_PLAYERS] if MAX_PLAYERS else all_players):
    pid = p['id']
    name = p['full_name']
    print(f"📌 Processing {i+1}: {name} (ID: {pid})")

    # Get basic game log
    log_obj = retry_request(playergamelog.PlayerGameLog, player_id=pid, season='2024-25')
    if log_obj is None:
        continue

    try:
        df_log = log_obj.get_data_frames()[0]
        if df_log.empty:
            continue

        avg = df_log[['PTS', 'AST', 'REB', 'STL', 'BLK', 'TOV', 'MIN',
                      'FG_PCT', 'FG3_PCT', 'FT_PCT', 'FGA', 'FG3A', 'FTA']].mean()

        # AST/TO ratio
        ast_to_ratio = avg['AST'] / avg['TOV'] if avg['TOV'] != 0 else None

        # True Shooting %
        ts_denominator = 2 * (avg['FGA'] + 0.44 * avg['FTA'])
        ts_pct = avg['PTS'] / ts_denominator if ts_denominator != 0 else None

    except Exception as e:
        print(f"❌ Failed to calculate averages for {name}: {e}")
        continue

    stats_data.append({
        'player_id': pid,
        'name': name,
        'avg_pts': avg['PTS'],
        'avg_ast': avg['AST'],
        'avg_reb': avg['REB'],
        'avg_stl': avg['STL'],
        'avg_blk': avg['BLK'],
        'avg_tov': avg['TOV'],
        'avg_min': avg['MIN'],
        'avg_fg_pct': avg['FG_PCT'],
        'avg_fg3_pct': avg['FG3_PCT'],
        'avg_ft_pct': avg['FT_PCT'],
        'avg_fga': avg['FGA'],
        'avg_fg3a': avg['FG3A'],      # ✅ 新增字段：三分出手数
        'avg_fta': avg['FTA'],
        'ast_to_ratio': ast_to_ratio,
        'ts_pct': ts_pct
    })

    time.sleep(0.6)

# Save to CSV
df_stats = pd.DataFrame(stats_data)
df_stats.to_csv("/Users/james/Desktop/player_stats_advanced.csv", index=False, quoting=1)
print("\n✅ Saved cleaned advanced player stats to Desktop.")


📌 Processing 1: Precious Achiuwa (ID: 1630173)
📌 Processing 2: Steven Adams (ID: 203500)
📌 Processing 3: Bam Adebayo (ID: 1628389)
📌 Processing 4: Ochai Agbaji (ID: 1630534)
📌 Processing 5: Santi Aldama (ID: 1630583)
📌 Processing 6: Trey Alexander (ID: 1641725)
📌 Processing 7: Nickeil Alexander-Walker (ID: 1629638)
📌 Processing 8: Grayson Allen (ID: 1628960)
📌 Processing 9: Jarrett Allen (ID: 1628386)
📌 Processing 10: Jose Alvarado (ID: 1630631)
📌 Processing 11: Kyle Anderson (ID: 203937)
📌 Processing 12: Giannis Antetokounmpo (ID: 203507)
📌 Processing 13: Cole Anthony (ID: 1630175)
📌 Processing 14: OG Anunoby (ID: 1628384)
📌 Processing 15: Taran Armstrong (ID: 1642379)
📌 Processing 16: Deni Avdija (ID: 1630166)
📌 Processing 17: Deandre Ayton (ID: 1629028)
📌 Processing 18: Marvin Bagley III (ID: 1628963)
📌 Processing 19: Patrick Baldwin Jr. (ID: 1631116)
📌 Processing 20: LaMelo Ball (ID: 1630163)
📌 Processing 21: Lonzo Ball (ID: 1628366)
📌 Processing 22: Mo Bamba (ID: 1628964)
📌 Proces